In [29]:
import Pkg
Pkg.activate("../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [30]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [31]:
include("VARs/utils.jl")
includet("bootstrap_model.jl")

In [35]:
#df = DataFrame(XLSX.readtable("../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
df = DataFrame(XLSX.readtable("../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

872×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬─────────┬────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E       │ CAPE  ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼─────────┼────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │    2.44 │ 12.52 ⋯
│ 1952-01-01 │  0.02463 │   104.771 │       23.75 │  1.41667 │ 2.42667 │ 12.36 ⋯
│ 1952-02-01 │  0.02617 │   103.585 │       23.81 │     1.42 │ 2.41333 │ 12.36 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │     2.4 │ 12.24 ⋯
│ 1952-04-01 │  0.02422 │   105.806 │       23.73 │     1.44 │    2.38 │ 12.20 ⋯
│ 1952-05-01 │  0.02384 │   106.373 │       24.38 │     1.45 │    2.36 │ 12.44 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │    2.34 │ 12.66 ⋯
│ 1952-07-01 │  0.02466 │   106.034 │       25.18 │     1.45 │ 2.34667 │ 12.67 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [55]:
freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
yeld_diff = diff(data_source[:Yeld_10Y])
start = Date(1988, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq)) 

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1232,1.3903,0.1048,0.3652,-2.6994,-1.5922,-0.0728,0.1903,7.6171,0.8504,0.1515
Price_10Y,0.0739,1.0968,0.053,0.2173,-0.902,-0.06,-0.096,0.1066,0.6796,0.7176,0.0452


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1234,1.3604,0.0775,0.3395,-2.5776,-1.4543,-0.0963,0.1805,6.5529,0.6278,0.1259
Price_10Y_CPI,0.0765,1.2215,0.0257,0.192,-0.9476,-0.0319,-0.1374,0.1496,0.96,0.336,0.0253


...

"Excess_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1232,1.3882,0.0758,0.3335,-2.7107,-1.5659,-0.1069,0.1885,7.4388,0.6151,0.131
Price_10Y_T_Bill,0.0734,1.0949,0.024,0.1887,-0.911,-0.0258,-0.1264,0.0969,0.7337,0.3265,0.0237


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0876963
 0.0876963  1.0

In [56]:
display(returns_summarystats(to(from(yeld_diff, start),end_d),1))

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0027,0.0095,-0.0001,0.0015,-0.0108,0.0189,-0.0017,0.0997,0.554,-0.0405,0.0


nothing

In [57]:
r = to(from(real_returns, start),end_d)
max_drawdown_and_length(values(r))

([0.5167442719307588, 0.3764979947356336], [153, 54])

In [58]:
data = to(from(real_returns, start),end_d)
returns1y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI,],12) ./1
returns5y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],60) ./5
returns10y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],120) ./10
returns30y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],360) ./30
cum_returns = merge(TimeSeries.rename!(returns1y, [:RealStock1y, :RealBonds1y, ]), 
                    TimeSeries.rename!(returns5y, [:RealStock5y, :RealBonds5y, ]),
                    TimeSeries.rename!(returns10y, [:RealStock10y, :RealBonds10y, ]),
                    TimeSeries.rename!(returns30y, [:RealStock30y, :RealBonds30y, ]))
returns_summarystats(expm1.(cum_returns),1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
RealStock1y,0.1477,0.463,0.101,0.198,-0.2128,-0.1996,0.0285,0.9047,-0.014,0.6841,0.1221
RealBonds1y,0.1066,0.2033,-0.0317,0.0533,-0.2337,0.2606,-0.1079,0.9586,-0.6835,-0.297,-0.0522
RealStock5y,0.0279,0.1541,0.1007,0.118,0.0508,0.2019,0.0785,0.9176,-0.869,3.6103,0.0988
RealBonds5y,0.0261,0.036,-0.0083,0.0115,-0.0566,-0.2034,-0.0324,0.9597,-1.1381,-0.3162,-0.0029
RealStock10y,0.0177,0.1427,0.108,0.1191,0.0742,0.1046,0.0942,0.9093,-0.8428,6.1061,0.1106
RealBonds10y,0.0187,0.0345,0.0038,0.0198,-0.0259,-0.2647,-0.0175,0.9682,-1.4988,0.2011,0.0088
RealStock30y,0.004,0.083,0.0753,0.0786,0.0682,0.162,0.0719,0.8886,-1.1935,19.0038,0.0752
RealBonds30y,0.0092,0.0431,0.0309,0.0381,0.0138,-0.5094,0.0204,0.9764,-1.3468,3.3801,0.0356


### Stationary bootstrap
- get singe block
- get multople blocs with static length
- get single blocs with random lengt
- get random block length
- gent random block lengt when, lengt is greater when left length

In [59]:
#assets_names = [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
#assets_names = [:Total_Stock_T_Bill, :Price_10Y_T_Bill]
data = to(from(real_returns[assets_names], start),end_d) 
#data = to(from(excess_returns[assets_names], start),end_d) 


439×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 1988-01-01 │       0.0300847 │     0.0418491 │
│ 1988-02-01 │       0.0274427 │    0.00920113 │
│ 1988-03-01 │      -0.0139734 │    -0.0255694 │
│ 1988-04-01 │      -0.0254302 │    -0.0158648 │
│ 1988-05-01 │       0.0538815 │    -0.0181189 │
│ 1988-06-01 │      -0.0072615 │     0.0277009 │
│ 1988-07-01 │      -0.0214601 │    -0.0162429 │
│ 1988-08-01 │       0.0123649 │   -0.00742173 │
│     ⋮      │        ⋮        │       ⋮       │
│ 2024-01-01 │       0.0349213 │      -0.01189 │
│ 2024-02-01 │       0.0258185 │    -0.0241356 │
│ 2024-03-01 │      -0.0140089 │     0.0036489 │
│ 2024-04-01 │       0.0231748 │    -0.0372919 │
│ 2024-05-01 │       0.0345217 │     0.0176731 │
│ 2024-06-01 │       0.0233252 │     0.0154399 │
│ 2024-07-01 │      -0.0155933 │     0.025

In [60]:
scenarios = block_bootstrap(values(data), 10_000; block_size=120, random_block=true)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,Price_10Y_CPI
1,0.0716,0.042
5,0.0703,0.0396
15,0.0682,0.0324
25,0.0685,0.0298


,Total_Stock_CPI,Price_10Y_CPI
1,0.1626,0.0686
5,0.076,0.023
15,0.0371,0.018
25,0.0269,0.0146


,Total_Stock_CPI,Price_10Y_CPI
1,-1.261,-0.3479
5,-0.174,-0.1544
15,-0.1519,-0.685
25,-0.1344,-0.3624


,Total_Stock_CPI,Price_10Y_CPI
1,1.9333,-0.416
5,-0.7983,-0.3528
15,-0.0989,0.0604
25,0.2158,-0.4633


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4952,-0.3677,-0.2486,0.0209,0.0947,0.1777,0.2667,0.3053,0.3592
5,-0.0844,-0.0596,-0.0485,-0.0007,0.0843,0.1236,0.1916,0.2062,0.2171
15,-0.0222,-0.0058,0.0087,0.0414,0.0707,0.0936,0.1279,0.1371,0.15
25,0.0013,0.0124,0.0236,0.0512,0.0684,0.0861,0.1125,0.1209,0.1299


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1225,-0.1007,-0.0871,-0.0044,0.0473,0.0959,0.1409,0.147,0.1808
5,-0.0095,-0.0048,-0.002,0.0227,0.0415,0.0552,0.0763,0.0826,0.089
15,-0.0126,-0.0094,-0.005,0.0235,0.0352,0.0447,0.0572,0.0604,0.0644
25,-0.0062,0.0014,0.0052,0.0185,0.032,0.041,0.051,0.0541,0.0573


In [61]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(quantile(mdd, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(quantile(mdd, [.03, .25, .5, .75, .97])')

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.342457  0.498531  0.516744  0.516744  0.664398

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.123824  0.131816  0.209137  0.376498  0.419367

In [62]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(quantile(mddl, [.03, .25, .5, .75, .97])')

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 37.0  71.0  121.0  153.0  237.0

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 37.0  54.0  73.0  132.0  255.0

In [63]:
#assets_names = [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
#assets_names = [:Total_Stock_T_Bill, :Price_10Y_T_Bill]
data = to(from(real_returns[assets_names], start),end_d) 
#data = to(from(excess_returns[assets_names], start),end_d) 

data_mean = mean(data) 
data = data .- values(data_mean)

data = data .+ [.045 .02]


439×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 1988-01-01 │       0.0686282 │     0.0597079 │
│ 1988-02-01 │       0.0659862 │       0.02706 │
│ 1988-03-01 │       0.0245701 │   -0.00771051 │
│ 1988-04-01 │       0.0131133 │    0.00199401 │
│ 1988-05-01 │       0.0924249 │  -0.000260102 │
│ 1988-06-01 │        0.031282 │     0.0455597 │
│ 1988-07-01 │       0.0170834 │    0.00161594 │
│ 1988-08-01 │       0.0509084 │     0.0104371 │
│     ⋮      │        ⋮        │       ⋮       │
│ 2024-01-01 │       0.0734648 │    0.00596887 │
│ 2024-02-01 │        0.064362 │   -0.00627674 │
│ 2024-03-01 │       0.0245346 │     0.0215077 │
│ 2024-04-01 │       0.0617183 │     -0.019433 │
│ 2024-05-01 │       0.0730652 │     0.0355319 │
│ 2024-06-01 │       0.0618687 │     0.0332988 │
│ 2024-07-01 │       0.0229502 │     0.042

In [64]:
scenarios = block_bootstrap(values(data), 10_000; block_size=120, random_block=true)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,Price_10Y_CPI
1,0.5346,0.2579
5,0.531,0.2546
15,0.5295,0.2471
25,0.5301,0.2444


,Total_Stock_CPI,Price_10Y_CPI
1,0.1629,0.0689
5,0.0771,0.023
15,0.0378,0.0179
25,0.0271,0.0144


,Total_Stock_CPI,Price_10Y_CPI
1,-1.2598,-0.3426
5,-0.2137,-0.1354
15,-0.1748,-0.7191
25,-0.0923,-0.3346


,Total_Stock_CPI,Price_10Y_CPI
1,1.9013,-0.3872
5,-0.7211,-0.328
15,0.0118,0.1579
25,0.1357,-0.5403


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0291,0.1092,0.2125,0.4834,0.557,0.6411,0.73,0.7721,0.8071
5,0.3741,0.3995,0.4134,0.4592,0.5452,0.5865,0.6504,0.6687,0.6809
15,0.4348,0.4523,0.4681,0.5032,0.5326,0.5543,0.5901,0.6002,0.6131
25,0.463,0.4748,0.4851,0.5124,0.5306,0.5478,0.5751,0.5827,0.5916


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.0918,0.1129,0.1284,0.2118,0.264,0.3117,0.3568,0.3699,0.3997
5,0.2049,0.2095,0.2132,0.2381,0.2564,0.2701,0.2918,0.2973,0.3044
15,0.2017,0.205,0.2094,0.2386,0.2499,0.2593,0.272,0.2748,0.2787
25,0.2123,0.2167,0.2202,0.2329,0.2465,0.2556,0.2655,0.2682,0.2715


In [11]:
quantile(rand(Geometric(1/60), 10_000), [0., .25, .5, .75, 1.])

5-element Vector{Float64}:
   0.0
  17.0
  41.0
  82.0
 498.0

In [12]:
#TODO 
#- compare real returns, excess returns, term i real tbill
#- calc drow down and drow down lengt for scenarios
#- est returns 1955-1980
#- calculate rolling windows (one year, 15 years) percentile
# - compere yeld diffs, inflation from bootstrap

gdy data length 120
n_steps = 60
stsrt_idx = 59

max_end_idx = 1


In [29]:
using Distributions, StatsBase

n = rand(Normal(.07, .15),100_000)
n2 = .07 .+ .15 * randn(100_000)
t3 = .07 .+ .15 * rand(TDist(3), 100_000)
t4 = .07 .+ .15 * rand(TDist(4), 100_000)
t5 = .07 .+ .15 * rand(TDist(5), 100_000)
println(round.(quantile(n, [.03, .25, .5, .75, .97]),digits=3))
println(round.(quantile(n2, [.03, .25, .5, .75, .97]),digits=3))
println(round.(quantile(t3, [.03, .25, .5, .75, .97]),digits=3))
println(round.(quantile(t4, [.03, .25, .5, .75, .97]),digits=3))
println(round.(quantile(t5, [.03, .25, .5, .75, .97]),digits=3))



[-0.21, -0.031, 0.07, 0.171, 0.351]
[-0.212, -0.031, 0.071, 0.172, 0.354]
[-0.374, -0.044, 0.07, 0.185, 0.514]
[-0.32, -0.041, 0.07, 0.182, 0.465]
[-0.289, -0.039, 0.07, 0.179, 0.431]


In [43]:
colnames(data_to_save)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [46]:
data_to_save = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)
data_to_save = data_to_save[:Yeld_10Y,:Dividend,:E,:CAPE,:T_Bill]
writetimearray(data_to_save, "from_shiller_q.csv")

In [ ]:
function max_drawdown_length(returns::Matrix{Float64})
    # returns: Matrix where each column is a scenario, each row a time step
    # Returns a vector of maximum drawdown lengths for each scenario
    
    n_scenarios = size(returns, 2)
    max_dd_lengths = zeros(Int, n_scenarios)
    
    for scenario in 1:n_scenarios
        # Calculate cumulative returns (log-sum for returns)
        cum_returns = cumsum(returns[:, scenario])
        # Convert to price path (assuming starting price = 1)
        prices = exp.(cum_returns)
        
        max_price = prices[1]  # Track historical peak
        current_dd_length = 0  # Current drawdown length
        max_dd_length = 0      # Maximum drawdown length for this scenario
        start_idx = 1          # Start index of current drawdown
        
        for t in 2:length(prices)
            if prices[t] >= max_price
                # New peak, reset drawdown
                max_price = prices[t]
                current_dd_length = 0
                start_idx = t
            else
                # In drawdown, increment length
                current_dd_length = t - start_idx + 1
                max_dd_length = max(max_dd_length, current_dd_length)
            end
        end
        
        max_dd_lengths[scenario] = max_dd_length
    end
    
    return max_dd_lengths
end

# Example usage
using Random

# Simulate 1000 scenarios with 252 time steps (e.g., 1 year of trading days)
Random.seed!(42)
n_steps = 252
n_scenarios = 1000
returns = randn(n_steps, n_scenarios) * 0.01  # Normal returns with 1% std dev

# Calculate maximum drawdown lengths
max_dd_lengths = max_drawdown_length(returns)

# Print summary statistics
println("Mean max drawdown length: ", mean(max_dd_lengths))
println("Median max drawdown length: ", median(max_dd_lengths))
println("Max max drawdown length: ", maximum(max_dd_lengths))
println("Min max drawdown length: ", minimum(max_dd_lengths))

Mean max drawdown length: 161.269
Median max drawdown length: 159.5
Max max drawdown length: 252
Min max drawdown length: 34


In [9]:
returns = collect([.2 .15 -.2 -.05 .25 .1 .05; .2 .15 -.2 -.05 .15 .1 .05]')

max_drawdown_length(returns)

2-element Vector{Int64}:
 3
 4

In [33]:
function max_drawdown_and_length(returns::Matrix{Float64})
    # returns: Matrix where each column is a scenario, each row a time step
    # Returns a tuple of vectors: (max_drawdowns, max_dd_lengths)
    
    n_scenarios = size(returns, 2)
    max_drawdowns = zeros(Float64, n_scenarios)
    max_dd_lengths = zeros(Int, n_scenarios)
    
    for scenario in 1:n_scenarios
        # Calculate cumulative returns and convert to price path
        cum_returns = cumsum(returns[:, scenario])
        prices = exp.(cum_returns)  # Assuming log-returns, starting price = 1
        
        max_price = prices[1]  # Track historical peak
        max_drawdown = 0.0     # Maximum drawdown (percentage)
        current_dd_length = 0  # Current drawdown length
        max_dd_length = 0      # Maximum drawdown length
        start_idx = 1          # Start index of current drawdown
        
        for t in 2:length(prices)
            if prices[t] >= max_price
                # New peak, reset drawdown
                max_price = prices[t]
                current_dd_length = 0
                start_idx = t
            else
                # In drawdown, update length and drawdown
                current_dd_length = t - start_idx + 1
                current_drawdown = (max_price - prices[t]) / max_price
                max_dd_length = max(max_dd_length, current_dd_length)
                max_drawdown = max(max_drawdown, current_drawdown)
            end
        end
        
        max_drawdowns[scenario] = max_drawdown
        max_dd_lengths[scenario] = max_dd_length
    end
    
    return (max_drawdowns, max_dd_lengths)
end

# Example usage
using Random, Statistics

# Simulate 1000 scenarios with 252 time steps
Random.seed!(42)
n_steps = 252
n_scenarios = 1000
returns = randn(n_steps, n_scenarios) * 0.01  # Normal returns with 1% std dev

# Calculate maximum drawdowns and lengths
max_drawdowns, max_dd_lengths = max_drawdown_and_length(returns)

# Print summary statistics
println("Maximum Drawdown Statistics:")
println("  Mean: ", round(mean(max_drawdowns) * 100, digits=2), "%")
println("  Median: ", round(median(max_drawdowns) * 100, digits=2), "%")
println("  Max: ", round(maximum(max_drawdowns) * 100, digits=2), "%")
println("  Min: ", round(minimum(max_drawdowns) * 100, digits=2), "%")
println("\nMaximum Drawdown Length Statistics:")
println("  Mean: ", round(mean(max_dd_lengths), digits=2), " time steps")
println("  Median: ", round(median(max_dd_lengths), digits=2), " time steps")
println("  Max: ", maximum(max_dd_lengths), " time steps")
println("  Min: ", minimum(max_dd_lengths), " time steps")

Maximum Drawdown Statistics:
  Mean: 17.14%
  Median: 16.44%
  Max: 44.77%
  Min: 5.41%

Maximum Drawdown Length Statistics:
  Mean: 161.27 time steps
  Median: 159.5 time steps
  Max: 252 time steps
  Min: 34 time steps


In [34]:
returns = collect([.2 .15 -.2 -.05 .25 .1 .05; .2 .15 -.2 -.05 .15 .1 .05]')
dd, mddl = max_drawdown_and_length(returns)
println(round.(dd, digits=4))
println(mddl)


[0.2212, 0.2212]
[3, 4]


In [23]:
round.([3.456, 2.543], digits=2)

2-element Vector{Float64}:
 3.46
 2.54